In [ ]:
!pip install ortools

In [ ]:
#UEFA AFC CAF Conmebol  Concacaf	Repechaje
#13   5   5   4         3         2
nombre_federacion = ['UEFA', 'AFC', 'CAF', 'Conmebol', 'Concacaf', 'Repechaje']
nombre_equipo = ['Bélgica', 'Francia', 'Inglaterra', 'España', 'Portugal',
                 'Países Bajos', 'Dinamarca', 'Alemania', 'Suiza', 'Croacia',
                 'Serbia', 'Polonia',
                 'UEFA1',
                 'Catar',
                 'Irán', 'Japón', 'Corea del Sur',
                 'Arabia Saudita',
                 'Senegal', 'Marruecos', 'Túnez', 'Camerún', 'Ghana',
                 'Brasil', 'Argentina',
                 'Uruguay',
                 'Ecuador',
                 'México', 'EEUU',
                 'Canadá',
                 'AFC/Conmebol', 'OFC/Concacaf']

for i,v in enumerate(nombre_equipo):
  print(i,v)

0 Bélgica
1 Francia
2 Inglaterra
3 España
4 Portugal
5 Países Bajos
6 Dinamarca
7 Alemania
8 Suiza
9 Croacia
10 Serbia
11 Polonia
12 UEFA1
13 Catar
14 Irán
15 Japón
16 Corea del Sur
17 Arabia Saudita
18 Senegal
19 Marruecos
20 Túnez
21 Camerún
22 Ghana
23 Brasil
24 Argentina
25 Uruguay
26 Ecuador
27 México
28 EEUU
29 Canadá
30 AFC/Conmebol
31 OFC/Concacaf


In [ ]:
from ortools.sat.python import cp_model

#crear CSP
model = cp_model.CpModel()

#variables y dominios
##variable g_i_j que significa que el equipo i está en el grupo j
grupos = []
for i in range(32): #para cada equipo
  grupos.append([model.NewBoolVar('grupo_'+str(i)+'_'+str(j)) for j in range(8)])
print(grupos)
##variable de federacion de cada equipo
federacion = [model.NewIntVar(1,6,'federacion_'+str(i)) for i in range(32)]
print(federacion)

##variable de bombos
bombo = [model.NewIntVar(1,4,'bombo_'+str(i)) for i in range(32)]
print(bombo)

#restricciones
#1    2   3   4         5         6
#UEFA AFC CAF Conmebol  Concacaf	Repechaje
#13   5   5   4         3         2
##distribuir federaciones
for i in range(32):
  if i < 13:
    model.Add(federacion[i] == 1) #UEFA
  elif i < 18:
    model.Add(federacion[i] == 2) #AFC
  elif i < 23:
    model.Add(federacion[i] == 3) #CAF
  elif i < 27:
    model.Add(federacion[i] == 4) #Conmebol
  elif i < 30:
    model.Add(federacion[i] == 5) #Concacaf
  else:
    model.Add(federacion[i] == 6) #Repechaje

##distribuir bombos
for i in range(32):
  if i in [0,1,2,3,4] + [13] + [23,24]:
    model.Add(bombo[i] == 1)
  elif i in [5,6,7,8,9] + [25] + [27,28]:
    model.Add(bombo[i] == 2)
  elif i in [10,11] + [14,15,16] + [18,19,20]:
    model.Add(bombo[i] == 3)
  else:
    model.Add(bombo[i] == 4)

##cada equipo participa en un solo grupo
for i in range(32):
  model.Add(sum(grupos[i]) == 1)

##cada grupo tiene 4 equipos
for j in range(8):
  model.Add(sum([grupos[i][j] for i in range(32)]) == 4)

##cada grupo tiene 4 equipos de federaciones distintas
grupos_federacion = []
grupos_bombo = []
for i in range(32):
  grupos_federacion.append([model.NewIntVar(0,1000,'gf_'+str(i)+'_'+str(j)) for j in range(8)])
  grupos_bombo.append([model.NewIntVar(0,1000,'gb_'+str(i)+'_'+str(j)) for j in range(8)])
print(grupos_federacion)
print(grupos_bombo)
##calcular las federaciones para los grupos
for i in range(32):
  for j in range(8):
    model.AddMultiplicationEquality(grupos_federacion[i][j],[federacion[i],grupos[i][j]])
    model.AddMultiplicationEquality(grupos_bombo[i][j],[bombo[i],grupos[i][j]])

#ejemplo count_vars(x=[1,2,3,4,1,5,8,1],val=1) -> c = 3
def count_vars(model, x, val, c): #c = cantidad de val en el arreglo x
  b = [model.NewBoolVar('b'+str(i)) for i in range(len(x))]
  for i in range(len(x)):
    #(x[i] == val) <==> b[i]
    ##b[i] => (x[i] == val)
    model.Add(x[i] == val).OnlyEnforceIf(b[i])
    ##(x[i] == val) => b[i] <----> not(b[i]) => not(x[i] == val)
    model.Add(x[i] != val).OnlyEnforceIf(b[i].Not())
  model.Add(c == sum(b))

count = {}
for j in range(8): #8 grupos
  for k in range(1,7): #6 federaciones
    count[(k,j)] = model.NewIntVar(0,1000,'count_'+str(k)+'_'+str(j))
    count_vars(model, [grupos_federacion[i][j] for i in range(32)], k, count[(k,j)])
    if k == 1: #UEFA
      model.Add(count[(k,j)] <= 2)
    elif k > 1: #el resto de federaciones
      model.Add(count[(k,j)] <= 1)

count_bombos = {}
for j in range(8): #8 grupos
  for k in range(1,5): #4 bombos
    count_bombos[(k,j)] = model.NewIntVar(0,1000,'cb_'+str(k)+'_'+str(j))
    count_vars(model, [grupos_bombo[i][j] for i in range(32)], k, count_bombos[(k,j)])
    model.Add(count_bombos[(k,j)] <= 1)

[[grupo_0_0(0..1), grupo_0_1(0..1), grupo_0_2(0..1), grupo_0_3(0..1), grupo_0_4(0..1), grupo_0_5(0..1), grupo_0_6(0..1), grupo_0_7(0..1)], [grupo_1_0(0..1), grupo_1_1(0..1), grupo_1_2(0..1), grupo_1_3(0..1), grupo_1_4(0..1), grupo_1_5(0..1), grupo_1_6(0..1), grupo_1_7(0..1)], [grupo_2_0(0..1), grupo_2_1(0..1), grupo_2_2(0..1), grupo_2_3(0..1), grupo_2_4(0..1), grupo_2_5(0..1), grupo_2_6(0..1), grupo_2_7(0..1)], [grupo_3_0(0..1), grupo_3_1(0..1), grupo_3_2(0..1), grupo_3_3(0..1), grupo_3_4(0..1), grupo_3_5(0..1), grupo_3_6(0..1), grupo_3_7(0..1)], [grupo_4_0(0..1), grupo_4_1(0..1), grupo_4_2(0..1), grupo_4_3(0..1), grupo_4_4(0..1), grupo_4_5(0..1), grupo_4_6(0..1), grupo_4_7(0..1)], [grupo_5_0(0..1), grupo_5_1(0..1), grupo_5_2(0..1), grupo_5_3(0..1), grupo_5_4(0..1), grupo_5_5(0..1), grupo_5_6(0..1), grupo_5_7(0..1)], [grupo_6_0(0..1), grupo_6_1(0..1), grupo_6_2(0..1), grupo_6_3(0..1), grupo_6_4(0..1), grupo_6_5(0..1), grupo_6_6(0..1), grupo_6_7(0..1)], [grupo_7_0(0..1), grupo_7_1(0..1)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
  #for i in range(32):
    #print(nombre_equipo[i],nombre_federacion[solver.Value(federacion[i])-1])
  #header
  print('\t',end='')
  for i in range(32):
    print(solver.Value(federacion[i]),end='\t')
  print()
  print('\t',end='')
  for i in range(32):
    print(i+1,end='\t')
  print()
  #grupos
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(grupos[i][j]),end='\t')
    print()
  print()
  #grupos_federacion
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(grupos_federacion[i][j]),end='\t')
    print()
  print()
  #grupos_bombos
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(grupos_bombo[i][j]),end='\t')
    print()
  print()
  #mostrar resultado
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      if solver.Value(grupos[i][j]):
        s = nombre_equipo[i] + '(' + nombre_federacion[solver.Value(federacion[i]) - 1] + ')'
        print(s + ' '*(20-len(s)),end='\t')
    [print(solver.Value(count[(k,j)]),end=' ') for k in range(1,7)]
    print(end='\t')
    [print(solver.Value(count_bombos[(k,j)]),end=' ') for k in range(1,5)]
    print()

	1	1	1	1	1	1	1	1	1	1	1	1	1	2	2	2	2	2	3	3	3	3	3	4	4	4	4	5	5	5	6	6	
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	
G1->	0	0	0	0	0	0	0	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	0	
G2->	0	0	0	0	0	0	0	0	1	0	0	0	1	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	
G3->	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	1	0	0	1	0	0	0	0	0	0	0	0	
G4->	1	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	
G5->	0	1	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	1	0	0	0	
G6->	0	0	0	1	0	0	0	1	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	
G7->	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	
G8->	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	1	0	0	0	0	0	1	0	0	0	0	

G1->	0	0	0	0	0	0	0	0	0	0	0	1	0	2	0	0	0	0	0	0	0	0	0	0	0	4	0	0	0	5	0	0	
G2->	0	0	0	0	0	0	0	0	1	0	0	0	1	0	2	0	0	0	0	0	0	0	0	0	4	0	0	0	0	0	0	0	
G3->	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	2	0	0	3	0	0	4	0	0	0	0	0	0	0	0	
G4->	1	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	3	0	0	0	0	0	0	0	0	0	0	0	6	
G5->

In [ ]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
  def __init__(self, variables):
    cp_model.CpSolverSolutionCallback.__init__(self)
    self.__variables = variables
    self.__solution_count = 0

  def on_solution_callback(self):
    self.__solution_count += 1
    for j in range(8):
      print('G'+str(j+1),end='->\t')
      for i in range(32):
        if self.Value(self.__variables[i][j]):
          s = nombre_equipo[i]
          print(s+' '*(20-len(s)),end='\t')
      print()
    print()

  def solution_count(self):
    return self.__solution_count

In [ ]:
solver = cp_model.CpSolver()
solution_printer = VarArraySolutionPrinter(grupos)
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True
# Solve.
status = solver.Solve(model, solution_printer)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
G5->	Catar               	Marruecos           	Uruguay             	AFC/Conmebol        	
G6->	Países Bajos        	Serbia              	Brasil              	OFC/Concacaf        	
G7->	Portugal            	Suiza               	Corea del Sur       	Ecuador             	
G8->	España              	Polonia             	Ghana               	México              	

G1->	Bélgica             	Dinamarca           	Irán                	OFC/Concacaf        	
G2->	Inglaterra          	Alemania            	Arabia Saudita      	Marruecos           	
G3->	Croacia             	Japón               	Camerún             	Argentina           	
G4->	Francia             	UEFA1               	Túnez               	EEUU                	
G5->	Catar               	Senegal             	Uruguay             	Canadá              	
G6->	Países Bajos        	Serbia              	Brasil              	AFC/Conmebol        	
G7->	Portugal            	Suiza 